# AML Coursework assessment

## Data Exploration

### Wisconsin Diagnostic Breast Cancer Dataset

The Wisconsin Diagnostic Breast Cancer dataset describes the measurements on cells in suspicious lumps in a women's breast. This dataset, obtained from the University of Wisconsin Hospitals, includes 30 feature variables and 1 target variable. 

The features, or input attributes, are numeric, and a binary target variable: M for Malignant and B for benign.

Each cell-nucleus include 10 actual features:

- Radius (mean of distances from center to points on the perimeter)
- Texture (standard deviation of gray-scale values)
- Perimeter
- Area
- Smoothness (local variation in radius lengths)
- Compactness (perimeter^2 / area - 1.0)
- Concavity (severity of concave portions of the contour)
- Concave points (number of concave portions of the contour)
- Symmetry 
- Fractal dimension ("coastline approximation" - 1)

Now, we will load the dataset so we can check the data in detail. To do so, we'll install the required libraries and packages.

In [ ]:
!pip3 install matplotlib pandas numpy seaborn scikit-learn

In [ ]:
!pip3 install -U ucimlrepo

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_diagnostic.data.features 
y = breast_cancer_wisconsin_diagnostic.data.targets 

data_combined = pd.concat([X, y], axis=1)
  
# metadata 
#print(breast_cancer_wisconsin_diagnostic.metadata) 
  
# variable information 
#print(breast_cancer_wisconsin_diagnostic.variables)



### Checking the data

We'll retrieve a data preview of the data so we can have a first look at it.

In [ ]:
# Print the first few rows of the data
print("\nFirst few rows of Features and Target combined")
print(data_combined.head())

### The dimensionality of the dataset

We can see that we're able to access the repository from UCIML and the data is loaded. Now we need to understand the dimensionality of the dataset in terms of number of rows, columns (features and target variable).

In [ ]:
# Print the shape of the data

print("Shape of X (features):", X.shape)
print("Shape of y (targets):", y.shape)

### Statistical Summary

Let's retrieve the statistical properties of each attribute in order to get some insights.

In [ ]:
pd.set_option('display.width', 100)
pd.set_option('display.precision', 3)

descriptionDataCombined = data_combined.describe()

print(descriptionDataCombined)

### Class Distribution

As this is a classification problem, we need to analyse the data in order to check how many observations do we have for each class. If they are highly imbalanced, then we'll need to take further actions in the data pre-processing.

In [ ]:
class_counts = data_combined.groupby('Diagnosis').size()
print(class_counts)

total = class_counts.sum()
proportion_B = class_counts['B'] / total
proportion_M = class_counts['M'] / total

print(f"\nProportion of class B: {proportion_B:.2f}")
print(f"Proportion of class M: {proportion_M:.2f}")

### Features Correlation 

By calculating the correlation between the different variables we'll be able to analyse if they are tightly coupled (highly correlated). This is important as it can affect the performance of the machine learning algorithms.

Let's calculate the Pearson's Correlation Coefficient, assuming that we have a normal distribution in the attributes. Before that, we need to convert the target variable to a numeric one.

In [ ]:
# Convert the Diagnosis column from string to numerical values.
data_combined['Diagnosis'] = data_combined['Diagnosis'].map({'M': 1, 'B': 0})

# Calculate the Pearson's Correlation Coefficient
correlations = data_combined.corr(method='pearson')
print(correlations)

As the table size is big, let's retrieve those variables that are highly correlated (> 0.75) by sorting and printing them.

In [ ]:
# Unstack the correlation matrix
correlation_pairs = correlations.unstack()

# Sort the correlation pairs
sorted_pairs = correlation_pairs.sort_values(kind="quicksort", ascending=False)

# Retrieve high correlations and exclude self-correlations, as they aren't relevant.
high_correlations = sorted_pairs[(sorted_pairs != 1.0) & (sorted_pairs.abs() > 0.75)]

print(high_correlations)

Let's plot a correlation matrix to achieve a better understanding of the data correlation

Datasource: https://stackoverflow.com/questions/29432629/plot-correlation-matrix-using-pandas

In [ ]:
# Plotting the Correlation Matrix

plt.figure(figsize=(10, 8))
plt.imshow(correlations, cmap='coolwarm', interpolation='nearest')
plt.colorbar(label='Correlation')
plt.title('Correlation Matrix')
plt.xticks(range(len(correlations.columns)), correlations.columns, rotation=90)
plt.yticks(range(len(correlations.index)), correlations.index)
plt.show()

We can see that the highly correlated variables are the radius and perimeter. This makes sense due to its mathematical relationship, as the radius is directly proportional to the perimeter.

### Skew of Univariate Distributions

Now we'll check if there is any attribute with a skew, in order to consider it during our data preparation phase.

In [ ]:
skew = data_combined.skew()

sorted_skew = skew.sort_values(ascending=False)

negative_skew = sorted_skew[sorted_skew < 0]
positive_skew = sorted_skew[sorted_skew > 0]
higher_skew = sorted_skew[sorted_skew > 1]

print("\nNegative Skew\n")
print(negative_skew)

print("\nPositive Skew\n")
print(positive_skew)

print("\nHigher Skew\n")
print(higher_skew)

The skew result shows there is no negative skew as all attribute values are positive. 
Additionally, we can see that some features have a higher skew than the rest of the features as they are greater than 3.
Now, we'll plot the histograms of each attribute in order to check the distribution of each of them.

In [ ]:
# Univariate Histograms
data_combined.hist(figsize=[20, 20])
plt.show()

## Data Transformation

We don't see the need of any major rescaling or transformation in the data. What is needed, is to transform the target column, as it isn't numeric.

In [54]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_numeric = label_encoder.fit_transform(y.squeeze())

# Print the transformed target variable to check if we have transformed it correctly
print("Transformed Target Variables (First 10 elements):", y_numeric[:10])

Unique Classes: ['B' 'M']
Numeric Codes: [0 1]
Transformed Target Variable (First 10 elements): [1 1 1 1 1 1 1 1 1 1]


## Feature Selection

### Principal Component Analysis (PCA)

As a preprocessing step, and in order to exclude irrelevant or redundant features, we will carry out a dimensionality reduction using the PCA technique. 

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)
fit = pca.fit(X)

# Get the explained variance ratios
explained_variance_ratios = fit.explained_variance_ratio_

# Transform the explained variance ratios into percentages
explained_variance_percentages = explained_variance_ratios * 100

# Print the explained variance percentages
for i, explained_variance_percentage in enumerate(explained_variance_percentages):
    print(f"Explained Variance for PC{i+1}: {explained_variance_percentage:.10f}%")

We see that the Principal Component 1 has a significantly higher variance than the rest so it makes sense to only keep it. Because of that, we will reduce the number of components to 2.

In [ ]:
pca = PCA(n_components=2)
fit = pca.fit(X)

# Get the explained variance ratios
explained_variance_ratios = fit.explained_variance_ratio_

# Transform the explained variance ratios into percentages
explained_variance_percentages = explained_variance_ratios * 100

# Print the explained variance percentages
for i, explained_variance_percentage in enumerate(explained_variance_percentages):
    print(f"Explained Variance for PC{i+1}: {explained_variance_percentage:.10f}%")

Now that we have two PCs, we will fit them to the original data

In [ ]:
X_pca = pca.fit_transform(X)

## Resampling

Since we already have the transformed dataset with the two selected components stored in the variable X_PCA, we will perform the resampling method K-Fold Cross-Validation to know how well the different models perform with data that haven´t been used to train them.

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

num_folds = 10
seed = 7

kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)

model = LogisticRegression(solver = 'liblinear')

results = cross_val_score(model, X, y, cv=kfold)

print("Accuracy: %.3f%% (%.3f%%)" % (results.mean() * 100.0, results.std() * 100.0))